In [ ]:
# Import required libraries
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap, PowerNorm

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Data Loading and Exploration

Let's start by loading the flight data and exploring its structure.

In [ ]:
# Define column names
csv_cols = ['dep_lat', 'dep_lon', 'arr_lat', 'arr_lon', 'nb_flights', 'CO2']

# Load the data
routes = pd.read_csv(
    '../data.csv',
    names=csv_cols,
    na_values=['\\N'],
    sep=';',
    skiprows=1
)

print(f"Total routes: {len(routes)}")
routes.head(10)

In [ ]:
# Basic statistics
print("\nDataset Statistics:")
print(routes.describe())

## 2. Data Analysis

Let's analyze the flight patterns.

In [ ]:
# Find busiest routes
print("Top 10 Busiest Routes:")
top_routes = routes.nlargest(10, 'nb_flights')
for _idx, route in top_routes.iterrows():
    print(f"  {route['nb_flights']:.0f} flights: "
          f"({route['dep_lat']:.2f}, {route['dep_lon']:.2f}) → "
          f"({route['arr_lat']:.2f}, {route['arr_lon']:.2f})")

In [ ]:
# Distribution of flight frequencies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
ax1.hist(routes['nb_flights'], bins=50, edgecolor='black')
ax1.set_xlabel('Number of Flights')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Flight Frequencies')

# Log scale
ax2.hist(routes['nb_flights'], bins=50, edgecolor='black', log=True)
ax2.set_xlabel('Number of Flights')
ax2.set_ylabel('Frequency (log scale)')
ax2.set_title('Distribution of Flight Frequencies (Log Scale)')

plt.tight_layout()
plt.show()

In [ ]:
# CO2 intensity analysis
fig, ax = plt.subplots(figsize=(12, 6))
scatter = ax.scatter(
    routes['nb_flights'],
    routes['CO2'],
    c=routes['nb_flights'],
    cmap='viridis',
    alpha=0.5,
    s=10
)
ax.set_xlabel('Number of Flights')
ax.set_ylabel('CO2 Intensity')
ax.set_title('Flight Frequency vs CO2 Intensity')
plt.colorbar(scatter, label='Number of Flights')
plt.show()

## 3. Basic Visualization

Create a simple scatter plot of departure and arrival locations.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Departure airports
ax1.scatter(routes['dep_lon'], routes['dep_lat'], alpha=0.3, s=1)
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('Departure Airports')
ax1.grid(True, alpha=0.3)

# Arrival airports
ax2.scatter(routes['arr_lon'], routes['arr_lat'], alpha=0.3, s=1, color='red')
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.set_title('Arrival Airports')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Advanced Map Visualization

Create a beautiful world map with flight routes using Cartopy.

In [ ]:
# Sample a subset for faster rendering in notebook
sample_size = 100
routes_sample = routes.nlargest(sample_size, 'nb_flights')

print(f"Visualizing top {sample_size} busiest routes")

In [ ]:
# Create the map
fig = plt.figure(figsize=(20, 12))
ax = plt.axes(projection=ccrs.Miller())

# Set global extent
ax.set_global()

# Add map features
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')

# Normalize colors by flight frequency
norm = PowerNorm(
    gamma=0.3,
    vmin=routes_sample['nb_flights'].min(),
    vmax=routes_sample['nb_flights'].max()
)

# Create colormap
color_list = [
    (0.8, 0.0, 0.6, 0.2),
    (0.8, 0.0, 0.6, 0.6),
    (1.0, 0.8, 0.9, 1.0)
]
cmap = LinearSegmentedColormap.from_list('routes', color_list)

# Plot routes
for _, route in routes_sample.iterrows():
    color_val = norm(route['nb_flights'])
    color = cmap(color_val)
    
    ax.plot(
        [route['dep_lon'], route['arr_lon']],
        [route['dep_lat'], route['arr_lat']],
        color=color,
        linewidth=1.0,
        transform=ccrs.Geodetic(),
        alpha=0.6
    )

ax.set_title(f'Top {sample_size} Busiest Flight Routes', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

## 5. Custom Styling

Create visualizations with different color schemes and projections.

In [ ]:
# Dark theme visualization
fig = plt.figure(figsize=(20, 12), facecolor='black')
ax = plt.axes(projection=ccrs.Robinson(), facecolor='black')

ax.set_global()

# Dark theme colors
ax.add_feature(cfeature.COASTLINE, color=(0.8, 0.0, 0.6, 0.7), linewidth=0.5)
ax.add_feature(cfeature.LAND, facecolor='black')
ax.add_feature(cfeature.OCEAN, facecolor='black')

# Neon color scheme
neon_colors = [
    (0.0, 0.0, 0.0, 0.0),
    (0.0, 1.0, 1.0, 0.5),
    (1.0, 0.0, 1.0, 1.0)
]
neon_cmap = LinearSegmentedColormap.from_list('neon', neon_colors)

# Plot sample routes
for i, (_, route) in enumerate(routes_sample.iterrows()):
    color = neon_cmap(i / len(routes_sample))
    
    ax.plot(
        [route['dep_lon'], route['arr_lon']],
        [route['dep_lat'], route['arr_lat']],
        color=color,
        linewidth=0.8,
        transform=ccrs.Geodetic(),
        alpha=0.7
    )

ax.set_title(
    'Flight Routes - Dark Theme (Robinson Projection)',
    fontsize=16,
    pad=20,
    color='white'
)
plt.tight_layout()
plt.show()

## 6. Regional Analysis

Focus on specific regions for detailed analysis.

In [ ]:
# Europe region
fig = plt.figure(figsize=(15, 12))
ax = plt.axes(projection=ccrs.PlateCarree())

# Set European extent
ax.set_extent([-15, 40, 35, 70], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, alpha=0.5)
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')

# Filter European routes
europe_routes = routes[
    (routes['dep_lon'].between(-15, 40)) & 
    (routes['dep_lat'].between(35, 70)) &
    (routes['arr_lon'].between(-15, 40)) & 
    (routes['arr_lat'].between(35, 70))
].nlargest(50, 'nb_flights')

for _, route in europe_routes.iterrows():
    ax.plot(
        [route['dep_lon'], route['arr_lon']],
        [route['dep_lat'], route['arr_lat']],
        color='red',
        linewidth=0.5,
        transform=ccrs.Geodetic(),
        alpha=0.5
    )

ax.set_title('European Flight Routes', fontsize=14)
plt.tight_layout()
plt.show()

print(f"Found {len(europe_routes)} routes within Europe")

## Summary

This notebook demonstrated:
- Loading and exploring flight route data
- Analyzing patterns in flight frequencies and CO2 emissions
- Creating various visualizations with different projections
- Customizing maps with color schemes
- Regional analysis for specific geographic areas

Feel free to experiment with different:
- Sample sizes
- Color schemes
- Map projections
- Regional filters
- Route selection criteria